In [1]:
# Extract and Load operation from SQL Server to Postgres with Python

#import needed libraries
#!pip install psycopg2
from sqlalchemy import create_engine # to interact with PostgreSQL
import pyodbc # to query the SQL Server
import pandas as pd # to carry out the data extract and data load part

import os # because username and password was store in the system environmente variable
# It is a good habit not to hard code your credentials in the script and store them separetely
# the aim is to protect the credentials from being exposed in the ETL script
# it can be used a configuration file or system environment variables

#get password from environmnet var
pwd = os.environ['PGPASS']
uid =  os.environ['PGUID']
#sql db details
driver = "{SQL Server Native Client 11.0}"
server = "localhost"
database = "AdventureWorksDW2019;"

#extract data from sql server
def extract():
    try:
        # define source connection
        src_conn = pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server + '\SQLEXPRESS' + 
                                  ';DATABASE=' + database + ';UID=' + uid + ';PWD=' + pwd)
        
        # cursor allow python code to execute sql command in a database session. It is bound to the connection
        src_cursor = src_conn.cursor()
        
        # execute query: gets the table name from the system schema in sql server. Just 6 tables was selected for don't have to process the entire schema.
        src_cursor.execute(""" select  t.name as table_name
        from sys.tables t where t.name in ('DimProduct','DimProductSubcategory','DimProductSubcategory','DimProductCategory','DimSalesTerritory','FactInternetSales') """)
        
        # to get the records
        src_tables = src_cursor.fetchall()
        
        # as the records are in a list format so we have to iterate over them
        for tbl in src_tables:
            #query and load save data to dataframe
            df = pd.read_sql_query(f'select * FROM {tbl[0]}', src_conn) # f'string? # [0] to get the table name as string
            load(df, tbl[0]) # tbl[0] = table name
            
    except Exception as e:
        print("Data extract error: " + str(e))
        
    finally:
        # close the connection once we are done
        src_conn.close()


#load data to postgres
def load(df, tbl):
    try:
        rows_imported = 0
        # the connection to postgres
        engine = create_engine(f'postgresql://{uid}:{pwd}@{server}:5432/AdventureWorks')
        print(f'importing rows {rows_imported} to {rows_imported + len(df)}... for table {tbl}')
        
        # save df to postgres
        df.to_sql(f'stg_{tbl}', engine, if_exists='replace', index=False) # if_exists='replace' == truncate and load technique
        rows_imported += len(df)
        
        # add elapsed time to final print out
        print("Data imported successful")
        
    except Exception as e:
        print("Data load error: " + str(e))



try:
    #call extract function
    extract()
except Exception as e:
    print("Error while extracting data: " + str(e))

importing rows 0 to 606... for table DimProduct
Data imported successful
importing rows 0 to 4... for table DimProductCategory
Data imported successful
importing rows 0 to 37... for table DimProductSubcategory
Data imported successful
importing rows 0 to 11... for table DimSalesTerritory
Data imported successful
importing rows 0 to 60398... for table FactInternetSales
Data imported successful


Hi Nabil, make sure you have the SQL Server driver installed on your machine, create the etl user with provided script. Also, add a rule in the firewall to allow connections to SQL Server port 1433.